<h2> LDA with static stopping

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit
from mne.decoding import CSP

current_directory = os.path.abspath('')

project_root = os.path.abspath(os.path.join(current_directory, '..', '..', '..', '..'))

sys.path.append(project_root)

print("ROOT:", project_root)
from Early_predict_UQ.data.make_dataset import make_data

# epoch tmin  = 2 and tmax = 6 , as the motor imagery task lasted in that time

def run_expanding_classification(subjects, initial_window_length, expansion_rate, sfreq,  prediction_time):
    scores_across_subjects = []
    current_person = 0
    for person in subjects:
        current_person += 1
        print("Person %d" % (person))
        subject= [person]
        epochs, labels = make_data(subject)
        epochs_train = epochs.copy()
        labels = epochs.events[:, -1] - 4
        epochs_data = epochs.get_data(copy=False)
        epochs_data_train = epochs_train.get_data(copy=False)

        cv = ShuffleSplit(2, test_size=0.2, random_state=42)
        cv_split = cv.split(epochs_data_train)
        scores_cv_splits = []

        lda = LinearDiscriminantAnalysis()
        csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
        current_cv = 0 
        for train_idx, test_idx in cv_split:
            current_cv += 1
            y_train, y_test = labels[train_idx], labels[test_idx]
            X_train = csp.fit_transform(epochs_data_train[train_idx], y_train)
            lda.fit(X_train, y_train)
            w_start = np.arange(0, epochs_data.shape[2] - initial_window_length, expansion_rate) 
            scores_across_epochs = []
            for epoch_idx in range(len(test_idx)):
                for n, window_start in enumerate(w_start):
                    window_length = initial_window_length + n * expansion_rate
                    X_test_window = csp.transform(epochs_data_train[test_idx][:, :,  window_start:(window_start + window_length)])
                    X_test_epoch_window = X_test_window[epoch_idx]
                    print("window_start:", window_start)
                    print("prediction_time:", prediction_time)
                    if window_start == prediction_time:
                        score = lda.score(X_test_epoch_window.reshape(1, -1), [y_test[epoch_idx]])
                        scores_across_epochs.append(score)
                        break
            if current_cv == 1:
                scores_cv_splits = np.array(scores_across_epochs)
            else:
                scores_cv_splits = np.vstack((scores_cv_splits,np.array(scores_across_epochs)))

        mean_scores_across_cv = np.mean(scores_cv_splits, axis=0)
        if current_person == 1:
            scores_across_subjects  = np.array(mean_scores_across_cv)
        else:
            scores_across_subjects = np.vstack((scores_across_subjects,np.array(mean_scores_across_cv)))

        mean_scores_across_subjects = np.mean(scores_across_subjects, axis=0)
        accuracy = np.mean(mean_scores_across_subjects)
    return accuracy, epochs, labels

if __name__ == "__main__":
    subjects = [1, 2, 3, 4, 5, 6, 7, 8, 9]  # 9 subjects
    sfreq = 250      

    initial_window_length = int(sfreq * 0.5)  
    expansion_rate = int(sfreq * 0.1)   

    epochs, labels = make_data([1]) # just to access the epoch.shape
    epochs_data = epochs.get_data(copy=False)
    prediction_times = np.arange(0, epochs_data.shape[2] - initial_window_length, expansion_rate)

    accuracy_array = []

    #something like this
    print("accuracy_array: ", accuracy_array)
    print(" prediction_times: ",  prediction_times)
    for current_pred_time, prediction_time in enumerate(prediction_times):
        print(f"prediction_time: {current_pred_time + 1} / {len(prediction_times)}")
        accuracy, epochs, labels = run_expanding_classification(subjects, initial_window_length, expansion_rate, sfreq, prediction_time)
        accuracy_array.append(accuracy)

    accuracy_array = np.array(accuracy_array)
    print("accuracy_array: ", accuracy_array)
